In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_WRITE_TOKEN')
!pip install vllm

# KoBERT movie classification

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import time
import argparse

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=26,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
dataset_dir = "data"
bertmodel, vocab = get_pytorch_kobert_model()

movie_data = pd.read_csv(dataset_dir + '/movie_data.csv')

data_list = []
for q, label in zip(movie_data['plot'], movie_data['genre_index'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=123)

In [ ]:
movie_data

In [ ]:
data_list[0]

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 10
learning_rate =  0.00005

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).cuda()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
start = time.time()
num_epochs = 10

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            metric_accuracy = loss.data.cpu().numpy()
            metric_loss = train_acc / (batch_id+1)
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, metric_accuracy, metric_loss))

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

    print("epoch {} test acc {}\n".format(e+1, test_acc / (batch_id+1)))


training_time = time.time() - start
print("training time : ", round(training_time, 2))

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    device = torch.device("cuda:0")
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        genre = None
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            genre_index = np.argmax(logits)

            if genre_index == 0:
                genre = "드라마"
            elif genre_index == 1:
                genre = "액션"
            elif genre_index == 2:
                genre = "코미디"
            elif genre_index == 3:
                genre = "애니메이션"
            elif genre_index == 4:
                genre = "멜로/로맨스"
            elif genre_index == 5:
                genre = "공포"
            elif genre_index == 6:
                genre = "다큐멘터리"
            elif genre_index == 7:
                genre = "범죄"
            elif genre_index == 8:
                genre = "스릴러"
            elif genre_index == 9:
                genre = "SF"
            elif genre_index == 10:
                genre = "판타지"
            elif genre_index == 11:
                genre = "모험"
            elif genre_index == 12:
                genre = "미스터리"
            elif genre_index == 13:
                genre = "가족"
            elif genre_index == 14:
                genre = "공연실황"
            elif genre_index == 15:
                genre = "에로"
            elif genre_index == 16:
                genre = "전쟁"
            elif genre_index == 17:
                genre = "뮤지컬"
            elif genre_index == 18:
                genre = "서스펜스"
            elif genre_index == 19:
                genre = "느와르"
            elif genre_index == 20:
                genre = "무협"
            elif genre_index == 21:
                genre = "서부"
            elif genre_index == 22:
                genre = "블랙코미디"
            elif genre_index == 23:
                genre = "서사"
            elif genre_index == 24:
                genre = "컬트"
            elif genre_index == 25:
                genre = "실험"


        print("prediction of genre : " + genre)

In [ ]:
sentence = "16만 팔로워를 가진 힙합 댄서 E.D.윈에게 첫눈에 반한 12살 소녀 미나는 운 좋게 오디션을 통과하고 E.D.윈의 댄스 크루에 들어간다. 공부와 달리 마음대로 되지 않는 몸에 인생 첫 좌절을 마주한 미나. 하지만 포기란 없다! 한때 춤으로 이름 좀 날렸던 할머니의 지도하에 남사친 마르쿠스와 비밀스러운 연습을 시작하는데… 함께라면 할 수 있어! ★오늘부터 댄싱퀸★"

predict(sentence)

# vLLM serving


```sh
vllm serve Llama-3.2-3B-Instruct --port 8877 --dtype float16 --max-model-len 8192 --api-key token-abc123
```


In [ ]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://34.47.229.169:8877/v1",
    api_key="token-abc123",
)

In [ ]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "Llama-3.2-3B-Instruct",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [ ]:
message_list = [{"role": "system", "content": "당신은 유저의 질문에 최대한 정확하고 풍부한 정보를 전달하는 assistant 이다. 답변은 항상 한국어로 공손하게 답변해줘."}]

while True:
    user_prompt = input("USER > ")
    if user_prompt.lower() == "quit":
        break
    message_list.append({"role": "user", "content": user_prompt})

    assistant = generate_response(message_list)
    message_list.append({"role": "assistant", "content": assistant})

USER >  python 에서 list 함수에 값을 추가하는 python코드 생성해줘.


Python에서 List 함수에 값을 추가하는 방법은 여러 가지가 있습니다. dưới đây는 몇 가지 방법입니다.

### 1. append() 함수

`append()` 함수는 list에 원소를 추가하는 데 사용됩니다.

```python
# 예시
my_list = [1, 2, 3]
my_list.append(4)
print(my_list)  # [1, 2, 3, 4]
```

### 2. extend() 함수

`extend()` 함수는 list에 원소가 있는 다른 list를 추가하는 데 사용됩니다.

```python
# 예시
my_list = [1, 2, 3]
my_list.extend([4, 5, 6])
print(my_list)  # [1, 2, 3, 4, 5, 6]
```

### 3. insert() 함수

`insert()` 함수는 list에 원소를 추가하는 데 사용됩니다. 원소가 특정 위치에 추가할 수 있습니다.

```python
# 예시
my_list = [1, 2, 3]
my_list.insert(1, 4)
print(my_list)  # [1, 4, 2, 3]
```

### 4. += 연산자

`+=` 연산자는 list를 추가하는 데 사용됩니다.

```python
# 예시
my_list = [1, 2, 3

inference time: 8.52300 sec 




USER >  quit


In [ ]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."
user_prompt = """#입력
[원본 prompt] = 테니스를 잘 칠 수 있는 방법은?

#처리
당신은 세계 최고의 prompt engineer로 행동합니다.
당신의 역할은 사용자가 제공한 prompt를 높은 품질의 prompt로 개선하는 일 입니다.
다음의 지침에 따라 [원본 prompt]를 재구성하여 [최적화된 prompt] 를 생성해주세요.

1. 의도 파악: 사용자의 궁극적인 목표와 의도를 명확히 이해하고 명시하세요.
2. 맥락 확립: 관련된 배경 정보, 주제의 범위, 필요한 전문 지식 수준을 설정하세요.
3. 구체성 강화: 모호한 표현을 피하고, 명확하고 구체적인 지시사항을 제공하세요.
4. 구조화: 응답의 형식, 구조, 섹션을 명확히 지정하여 체계적인 출력을 유도하세요.
5. 예시 제공: 가능한 경우, 기대하는 출력의 구체적인 예시나 셈플 데이터를 포함하세요.
6. 제한 설정: 응답의 길이, 복잡성 수준, 사용할 기술 용어의 범위 등을 명시하세요.
7. 다각적 접근: 다양한 관점, 시나리오, 또는 해결 방법을 고려하도록 요청하세요.
8. 윤리적 고려: 편함, 민감한 내용, 또는 잠재적인 부작용에 대한 주의사항을 포함하세요.
9. 상호작용 유도: 필요한 경우, 추가 정보나 명확화를 요청할 수 있는 옵션을 제공하세요.
10. 품질 기준 설정: 응답을 평가할 수 있는 구체적인 기준이나 체크리스트를 포함하세요.

#출력
반드시 최적화된 prompt 만 출력합니다.
[최적화된 prompt]

"""

In [ ]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
generate_response(messages)

테니스를 잘 칠 수 있는 방법은?

1. 의도 파악: 사용자의 궁극적인 목표와 의도를 명확히 이해하고 명시하세요. 테니스를 잘 칠 करन다는 것은 테니스-ball을 적절히挥이기 위한 기술을 mastering하는 것이 아니라, 테니스 장면을 일상생활에서 효과적으로 표현하는 것이 목표입니다.
2. 맥락 확립: 관련된 배경 정보, 주제의 범위, 필요한 전문 지식 수준을 설정하세요. 테니스 장면을 그려야 하는 배경은 테니스 경기 또는 테니스 연습이며, 전문 지식 수준은 테니스 기술, 테니스 장면, 그려야 하는 형식(예: drawing, painting, sketching) 등이 포함됩니다.
3. 구체성 강화: 모호한 표현을 피하고, 명확하고 구체적인 지시사항을 제공하세요. 테니스를 잘 칠하는 방법은 테니스-ball을挥이기 위한 기술, 테니스 장면을 일상생활에서 효과적으로 표현하는 방법, 테니스 장면을 그려야 하는 형식 등이 포함됩니다.
4. 구조화: 응답의 형식, 구조, 섹션을 명확히 지정하여 체계적인 输出을 유도하세요. 테니스를 잘 칠하는 방법은 다음과 같이 구체화할 수 있습니다:
   - 테니스 기술을 mastering하는 방법
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법
   - 테니스 장면을 그려야 하는 형식
   - 테니스 장면을 그리는 기술
5. 예시 제공: 가능한 경우, 기대하는 输出의 구체적인 예시나 셈플 데이터를 포함하세요. 테니스를 잘 칠하는 방법은 다음과 같은 예시로 설명할 수 있습니다:
   - 테니스-ball을挥이기 위한 기술
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법
   - 테니스 장면을 그려야 하는 형식
   - 테니스 장면을 그리는 기술
6. 제한 설정: 응답의 길이, 복잡성 수준, 사용할 기술 용어의 범위 등을 명시하세요. 테니스를 잘 칠하는 방법은 다음과 같이 제한할 수 있습니다:
   - 길이: 500자 이상, 1000자 이하
   - 복잡성 수준: 중급
   - 기술 용어의 범위: 테니스 기술, 

'테니스를 잘 칠 수 있는 방법은?\n\n1. 의도 파악: 사용자의 궁극적인 목표와 의도를 명확히 이해하고 명시하세요. 테니스를 잘 칠 करन다는 것은 테니스-ball을 적절히挥이기 위한 기술을 mastering하는 것이 아니라, 테니스 장면을 일상생활에서 효과적으로 표현하는 것이 목표입니다.\n2. 맥락 확립: 관련된 배경 정보, 주제의 범위, 필요한 전문 지식 수준을 설정하세요. 테니스 장면을 그려야 하는 배경은 테니스 경기 또는 테니스 연습이며, 전문 지식 수준은 테니스 기술, 테니스 장면, 그려야 하는 형식(예: drawing, painting, sketching) 등이 포함됩니다.\n3. 구체성 강화: 모호한 표현을 피하고, 명확하고 구체적인 지시사항을 제공하세요. 테니스를 잘 칠하는 방법은 테니스-ball을挥이기 위한 기술, 테니스 장면을 일상생활에서 효과적으로 표현하는 방법, 테니스 장면을 그려야 하는 형식 등이 포함됩니다.\n4. 구조화: 응답의 형식, 구조, 섹션을 명확히 지정하여 체계적인 输出을 유도하세요. 테니스를 잘 칠하는 방법은 다음과 같이 구체화할 수 있습니다:\n   - 테니스 기술을 mastering하는 방법\n   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법\n   - 테니스 장면을 그려야 하는 형식\n   - 테니스 장면을 그리는 기술\n5. 예시 제공: 가능한 경우, 기대하는 输出의 구체적인 예시나 셈플 데이터를 포함하세요. 테니스를 잘 칠하는 방법은 다음과 같은 예시로 설명할 수 있습니다:\n   - 테니스-ball을挥이기 위한 기술\n   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법\n   - 테니스 장면을 그려야 하는 형식\n   - 테니스 장면을 그리는 기술\n6. 제한 설정: 응답의 길이, 복잡성 수준, 사용할 기술 용어의 범위 등을 명시하세요. 테니스를 잘 칠하는 방법은 다음과 같이 제한할 수 있습니다:\n   - 길이: 500자 이상, 1000자 이하\n   - 복잡성 수준: 중급\n   - 

In [ ]:
user_prompt = """테니스를 잘 칠 수 있는 방법알려줘.
답변할땐 아래의 task 를 reasoning 한 결과를 답변으로 제공해줘.
task:
1. 의도 파악: 사용자의 궁극적인 목표와 의도를 명확히 이해하고 명시하세요. 테니스를 잘 칠 करन다는 것은 테니스-ball을 적절히挥이기 위한 기술을 mastering하는 것이 아니라, 테니스 장면을 일상생활에서 효과적으로 표현하는 것이 목표입니다.
2. 맥락 확립: 관련된 배경 정보, 주제의 범위, 필요한 전문 지식 수준을 설정하세요. 테니스 장면을 그려야 하는 배경은 테니스 경기 또는 테니스 연습이며, 전문 지식 수준은 테니스 기술, 테니스 장면, 그려야 하는 형식(예: drawing, painting, sketching) 등이 포함됩니다.
3. 구체성 강화: 모호한 표현을 피하고, 명확하고 구체적인 지시사항을 제공하세요. 테니스를 잘 칠하는 방법은 테니스-ball을挥이기 위한 기술, 테니스 장면을 일상생활에서 효과적으로 표현하는 방법, 테니스 장면을 그려야 하는 형식 등이 포함됩니다.
4. 구조화: 응답의 형식, 구조, 섹션을 명확히 지정하여 체계적인 输出을 유도하세요. 테니스를 잘 칠하는 방법은 다음과 같이 구체화할 수 있습니다:
   - 테니스 기술을 mastering하는 방법
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법
   - 테니스 장면을 그려야 하는 형식
   - 테니스 장면을 그리는 기술
5. 예시 제공: 가능한 경우, 기대하는 输出의 구체적인 예시나 셈플 데이터를 포함하세요. 테니스를 잘 칠하는 방법은 다음과 같은 예시로 설명할 수 있습니다:
   - 테니스-ball을挥이기 위한 기술
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법
   - 테니스 장면을 그려야 하는 형식
   - 테니스 장면을 그리는 기술
6. 제한 설정: 응답의 길이, 복잡성 수준, 사용할 기술 용어의 범위 등을 명시하세요. 테니스를 잘 칠하는 방법은 다음과 같이 제한할 수 있습니다:
   - 길이: 500자 이상, 1000자 이하
   - 복잡성 수준: 중급
   - 기술 용어의 범위: 테니스 기술, 테니스 장면, 그려야 하는 형식
7. 다각적 접근: 다양한 관점, 시나리오, 또는 해결 방법을 고려하도록 요청하세요. 테니스를 잘 칠하는 방법은 다음과 같이 다각적으로 접근할 수 있습니다:
   - 테니스 기술을 mastering하는 방법
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법
   - 테니스 장면을 그려야 하는 형식
   - 테니스 장면을 그리는 기술
8. 윤리적 고려: 편함, 민감한 내용, 또는 잠재적인 부작용에 대한 주의사항을 포함하세요. 테니스를 잘 칠하는 방법은 다음과 같이 윤리적으로 고려할 수 있습니다:
   - 테니스 장면을 그려야 하는 형식에 대한 선택
   - 테니스 장면을 그리는 기술에 대한 선택
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법에 대한 선택
9. 상호작용 유도: 필요한 경우, 추가 정보나 명확화를 요청할 수 있는 옵션을 제공하세요. 테니스를 잘 칠하는 방법은 다음과 같이 상호작용을 유도할 수 있습니다:
   - 테니스 기술을 mastering하는 방법에 대한 추가 정보
   - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법에 대한 추가 정보
   - 테니스 장면을 그려야 하는 형식에 대한 추가 정보
10. 품질 기준 설정: 응답을 평가할 수 있는 구체적인 기준이나 체크리스트를 포함하세요. 테니스를 잘 칠하는 방법은 다음과 같이 품질 기준을 설정할 수 있습니다:
    - 테니스 기술을 mastering하는 방법
    - 테니스 장면을 일상생활에서 효과적으로 표현하는 방법
    - 테니스 장면을 그려야 하는 형식
    - 테니스 장면을 그리는 기술"""

In [ ]:
user_prompt = """아래의 영화 줄거리를 기반으로 해당 영화가 어떤 장르인지 설명해줘.
영화 줄거리:
세계 최고 바둑 대회에서 국내 최초 우승자가 된 조훈현. 전 국민적 영웅으로 대접받던 그는 바둑 신동이라 불리는 이창호를 제자로 맞는다. “실전에선 기세가 8할이야” 제자와 한 지붕 아래에서 먹고 자며 가르친 지 수년. 모두가 스승의 뻔한 승리를 예상했던 첫 사제 대결에서 조훈현은 전 국민이 지켜보는 가운데, 기세를 탄 제자에게 충격적으로 패한다. 오랜만에 패배를 맛본 조훈현과 이제 승부의 맛을 알게 된 이창호. 조훈현은 타고난 승부사적 기질을 되살리며 다시 한번 올라갈 결심을 하게 되는데"""

In [ ]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
generate_response(messages)

이 영화는 스포츠드라마 장르입니다.

inference time: 0.45928 sec 




'이 영화는 스포츠드라마 장르입니다.'